## Under development

## Levenberg-Marquardt

This fitter is identitied as ap.fit.LM and it employs a variant of the second order Newton's method to converge very quickly to the local minimum. This is the generally accepted best algorithm for most use cases in Chi^2 minimization. If you don't know what to pick, start with this minimizer. The LM optimizers bridges the gap between first-order gradient descent and second order Newton's method. When far from the minimum, Newton's method is unstable and can give wildly wrong results, however, near the minimum it has "quadratic convergence." This means that once near the minimum it takes only a few iterations to converge to several decimal places. The "L" scale parameter goes from L >> 1 which represents gradient descent to L << 1 which is Newton's Method. 

## Iterative Fit

An iterative fitter is identified as ap.fit.Iter and this makes use of the other fitters under certain circumstances. This method is generally employed for large models where it is not feasible to hold all the relevant data in memory at once. The iterative fitter will cycle through the models in a Group_Model object and fit them one at a time to the image, using the residuals from the previous cycle. This can be a very robust way to deal with some fits, especially if the overlap between models is not too strong. It is however more dependent on good initialization than other methods like the Levenberg-Marquardt. Also, it is possible for the Iter method to get stuck under certaint circumstances.

Note that while the Iterative fitter needs a Group_Model object to iterate over, it is not necessarily true that the sub models are BaseModel objects, they could be Group_Model objects as well. In this way it is possible to cycle through and fit "clusters" of objects that are nearby, so long as it doesn't consume too much memory.

## Gradient Descent

A gradient descent fitter is identified as ap.fit.Grad and uses standard first order derivative methods as provided by pytorch. These gradient descent methods include Adam, classic plus momentum, and LBFGS to name a few. The first order gradient is faster to evaluate and uses less memory, however it is considerably slower to converge than Levenberg-Marquardt. The gradient descent method with a small learning rate will reliably converge towards a local minimum, it will just do so slowly.

## Metropolis Adjusted Langevin Algorithm (MALA)

A MALA fitter is identified as ap.fit.MALA and implements a gradient based variant of Metropolis Hastings Markov-Chain Monte-Carlo. As an MCMC algorithm this will eventually converge to the global minimum of the Chi^2 surface though it is the slowest of all the algorithms by a large margin. There are many great resources out there for interpretting MCMC chains and the reader may wish to investigate those before attempting MCMC on an AutoProf model. Since AutoProf is built for very large models, this means a very large number of parameters which typically greatly increases the convergence time for the MCMC chains. It is possible to circumvent this "curse-of-dimensionality" to some extent using the MALA. The core idea of the MALA is that the proposal distribution (typically a multivariate gaussian) can be shifted in the direction of the negative gradient instead of centered on the current point. Extra work must be done the ensure detailed balance, but for large models it is worth the extra computational cost for a higher acceptance rate. For further reading see Hamiltonian Monte-Carlo (HMC) models of which MALA is essentially a single step version of the HMC.

NOTE: the MALA is still in development and not ready yet.